In [ ]:
"""
Update Parameters Here
"""
COLLECTION_NAME = "WomenRise"
CONTRACT = "0x7f7685b4cc34bd19e2b712d8a89f34d219e76c35"
CHAIN = "eth"

""" 
Optional parameters
"""
FOLDER = 1

KEEP_ALL_DATA = False  # set to TRUE to keep the raw JSON on disk

MAX_RESULTS = 500  # max results per request
TIME_DELTA = 1  # time to wait between successful calls
TIME_DELTA_2 = 5  # time to wait after API throttling message

In [2]:
"""
A Moralis API Key is required.
The free tier includes one and is enough to get minting data.

AVAILABLE CHAINS:
    eth, ropsten, rinkeby, goerli, kovan,
    polygon, mumbai, bsc, bsc testnet,
    avalanche, avalanche testnet, fantom
"""

import os
import requests
import json
import time
import pandas as pd
from pandas import json_normalize

from utils import config


def get_mintdata(
    COLLECTION_NAME,
    CONTRACT,
    CHAIN,
):

    RARITY_CSV = f"{config.RARITY_FOLDER}/{COLLECTION_NAME}_raritytools.csv"

    print(f"Rarity data loaded from: {RARITY_CSV}")
    RARITY_DB = pd.read_csv(RARITY_CSV)

    headers = {"Content-type": "application/json", "x-api-key": config.MORALIS_API_KEY}

    print(f"Getting minting data for {COLLECTION_NAME}")
    more_results = True
    page = 0
    start_time = time.time()
    all_data = list()  # empty list to store data as it comes
    while more_results:
        url = "https://deep-index.moralis.io/api/v2/nft/{}/transfers?chain={}&format=decimal&offset={}&limit={}".format(
            CONTRACT, CHAIN, page * 500, MAX_RESULTS
        )
        print(f"getting page {page + 1} ...")

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            print(
                "Sucessfuly received page {} of {}".format(
                    page + 1, int(1 + response.json()["total"] / MAX_RESULTS)
                )
            )
            PATH = (
                f"{config.MINTING_FOLDER}/{COLLECTION_NAME}/{page * MAX_RESULTS}.json"
            )

            # add new data to existing list
            all_data.extend(response.json()["result"])

            page += 1

            # if results in this response is less than MAX_RESULTS then it's the last page
            if len(response.json()["result"]) < MAX_RESULTS:
                more_results = False
            else:
                time.sleep(TIME_DELTA)

        elif response.status_code in [429, 503, 520]:
            print(
                f"Got a {response.status_code} response from the server. Waiting {TIME_DELTA_2} seconds and retrying"
            )
            time.sleep(TIME_DELTA_2)

        else:
            print(f"status_code = {response.status_code}")
            print("Received a unexpected error from Moralis API. Closing process.")
            more_results = False

    # Save full json data to one master file
    if KEEP_ALL_DATA:
        folder = f"{config.MINTING_FOLDER}/{COLLECTION_NAME}/"
        if not os.path.exists(folder):
            os.mkdir(folder)

        PATH = f"{config.MINTING_FOLDER}/{COLLECTION_NAME}/{COLLECTION_NAME}.json"
        with open(PATH, "w") as destination_file:
            json.dump(all_data, destination_file)

    df = json_normalize(all_data)
    # remove non minting rows
    df = df.loc[df["from_address"] == "0x0000000000000000000000000000000000000000"]

    # make sure token_id is an integer
    df["token_id"] = df["token_id"].astype(int)

    # add rarity rank to minting data
    df = df.merge(RARITY_DB, left_on="token_id", right_on="TOKEN_ID")

    # discard unwanted columns
    df = df[
        [
            "transaction_hash",
            "to_address",
            "token_id",
            "from_address",
            "Rank",
            "block_timestamp",
        ]
    ]

    df.drop_duplicates(subset=["token_id"], inplace=True)

    # get matching columns names to HonestNFT csv format
    df.columns = ["txid", "to_account", "TOKEN_ID", "current_owner", "rank", "time"]

    # clean 'time' field to make it compatible with the csv produced by 'find_minting_data.ipynb'
    df["time"] = df["time"].str.replace(".000Z", "")

    df.to_csv(f"{config.MINTING_FOLDER}/{COLLECTION_NAME}_minting.csv")

    print("--- %s seconds ---" % (round(time.time() - start_time, 1)))
    print("finished")


get_mintdata(COLLECTION_NAME, CONTRACT, CHAIN)

Rarity data loaded from: /home/flyingfish/honestnft-shenanigans-1/data/rarity_data/WomenRise_raritytools.csv
Getting minting data for WomenRise
getting page 1 ...
Sucessfuly received page 1 of 52
getting page 2 ...
Sucessfuly received page 2 of 52
getting page 3 ...
Sucessfuly received page 3 of 52
getting page 4 ...
Sucessfuly received page 4 of 52
getting page 5 ...
Sucessfuly received page 5 of 52
getting page 6 ...
Sucessfuly received page 6 of 52
getting page 7 ...
Sucessfuly received page 7 of 52
getting page 8 ...
Sucessfuly received page 8 of 52
getting page 9 ...
Sucessfuly received page 9 of 52
getting page 10 ...
Sucessfuly received page 10 of 52
getting page 11 ...
Sucessfuly received page 11 of 52
getting page 12 ...
Sucessfuly received page 12 of 52
getting page 13 ...
Sucessfuly received page 13 of 52
getting page 14 ...
Got a 429 response from the server. Waiting 2 seconds and retrying
getting page 14 ...
Got a 429 response from the server. Waiting 2 seconds and retryin

KeyboardInterrupt: 